In [3]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
import enchant


# English

In [2]:
data=pd.read_pickle('en_emails_raw.pkl')
# data=data[['class','message']].drop_duplicates()
data

,class,contains_img,email_from,email_to,message,subject,subpath
1,spam,False,"""Stella Lowry"" <rookcuduq@yahoo.com>","""Brian"" <bernice@groucho.cs.psu.edu>",\n ...,re[12]:,../data/000/001
2,spam,False,"""Walter"" <trwmpca@downtowncumberland.com>",<arline@groucho.cs.psu.edu>,Academic Qualifications available from prestig...,Take a moment to explore this.,../data/000/002
3,ham,False,Scott Schwartz <schwartz@groucho.cs.psu.edu>,9fans <plan9-fans@cs.psu.edu>,Greetings all. This is to verify your subscri...,Greetings,../data/000/003
4,spam,False,"""Mr Jailyn Koepke"" <kiflsbizc@attheworld.com>",melvin@groucho.cs.psu.edu,try chauncey may conferred the luscious not co...,LOANS @ 3.17% (27 term),../data/000/004
5,ham,False,Scott Schwartz <schwartz@groucho.cs.psu.edu>,9fans <plan9-fans@cs.psu.edu>,"It's quiet. Too quiet. Well, how about a str...",who wants to start?,../data/000/005
...,...,...,...,...,...,...,...
37817,spam,False,"""Christy Armstrong"" <contact@10-million-hits.com>",webmastr@kukui.ifa.hawaii.edu,Great News Expec ted!\n\nInfinex Ventures Inc....,RE: Watcher daily news,../data/126/017
37818,spam,False,"""Bette Ortiz"" <confirm@1000land.com>",webmastr@KUKUI.IFA.HAWAII.EDU,The OIL sector is going crazy. This is our wee...,RE: Number 1 market Picks,../data/126/018
37819,spam,False,"""Bettye Rowland"" <baitommyf@0451.com>",webmastr@KUKUI.IFA.HAWAII.EDU,http://vdtobj.docscan.info/?23759301\n\nSuffer...,All products for your health!,../data/126/019
37820,spam,False,"""Daisy"" <nluwwj@avk-pl.nl>",<webmastr@KUKUI.IFA.HAWAII.EDU>,U N I V E R S I T Y D I P L O M A S\n\nDo you...,"Its here, now.",../data/126/020


In [3]:
# data['message'].apply(lambda x: 'zipping my lean muscle mass' in x)
# temp=data[data['message'].apply(lambda x: 'zipping my lean muscle mass' in x)]
# temp.nunique()

In [4]:
en_regex = re.compile('[^a-zA-Z ]')
d = enchant.Dict("en_US") 
STOPWORDS = stopwords.words('english')

In [5]:
def enPreprocess(string):
    string=string.replace('\n',' ') #remove newline char
    string=en_regex.sub('',string) #removes non alphabets
    string=string.split()
    string=[word.lower() for word in string if len(word)>1]
    string=' '.join([word for word in string if d.check(word) and (word not in STOPWORDS)]) # split the string into list and check each word if it is in the english dictionary and longer than 1 alphabet
    return string

In [6]:
data['clean_msg']=data['message'].apply(lambda x: enPreprocess(x))
data.to_pickle('lowercase words.pkl')

In [8]:
from collections import Counter

words = data[data['class']=='spam']['clean_msg'].apply(lambda x: [word.lower() for word in x.split()])
ham_words = Counter()

for msg in words:
    ham_words.update(msg)
    
print(ham_words.most_common(50))

[('price', 12433), ('adobe', 10354), ('info', 8632), ('company', 8444), ('gold', 7831), ('professional', 7224), ('windows', 6447), ('us', 6095), ('office', 5812), ('campaign', 5395), ('website', 5178), ('stock', 5011), ('save', 4988), ('add', 4987), ('china', 4957), ('one', 4805), ('hi', 4796), ('reviews', 4743), ('retail', 4684), ('ms', 4655), ('get', 4615), ('cart', 4574), ('rating', 4544), ('please', 4404), ('pro', 4274), ('corp', 3979), ('may', 3963), ('effects', 3847), ('energy', 3759), ('product', 3719), ('news', 3649), ('new', 3618), ('weight', 3607), ('like', 3560), ('inc', 3425), ('offer', 3380), ('day', 3319), ('see', 3232), ('body', 3171), ('even', 3160), ('opt', 3074), ('also', 2986), ('back', 2976), ('best', 2893), ('development', 2880), ('email', 2874), ('oil', 2870), ('great', 2818), ('side', 2806), ('today', 2794)]


In [6]:
data=pd.read_pickle('lowercase words.pkl')

In [64]:
data.groupby('clean_msg').count().sort_values('class')


,class
clean_msg,
john office prepared poster announcing awards planning poster display entered building easel informed left overnight would likely stolen mind please recommend location poster mounted wall posters students others receiving external organizations location enter building suggestions please director foundation government grants university hall fax john office prepared poster announcing awards planning poster display entered building easel informed left overnight would likely stolen mind please recommend location poster mounted wall posters students others receiving external organizations location enter building suggestions please director foundation government grants university hall fax,1
please remove address mailing list dr warren jr associate dean academic programs university college engineering hall box fl email forwarding service brought let become free minutes,1
please remove list also department biology institute technology avenue phone fax email,1
please remove list also stop blind coping every removal request sent ms coordinator international admissions university south sc phone fax web address email forwarding service brought let become free minutes,1
please remove list interested messages,1
...,...
luxury watches buy frank muller omega tag full gold men fast delivery lowest prices world worldwide shipping visit shop,204
great news ted ventures inc price already started climb one well last marketing campaign well overview aggressive energetic boasts dynamic diversified portfolio operations across north eye international expansion grounded natural resource exploration also offers investors access exciting new developments sector booming international real estate market market based experience tenacious research techniques razor sharp analytical skills allow us leverage opportunities emerging markets developing technologies identifying opportunities earliest stages allows us accelerate business development fully realize true potential maximizing overall profitability turn enhancing shareholder value current press release announces joint venture option agreement extension may ventures inc board directors please announce company granted extension days fulfill contractual obligations joint venture option agreement dated island yew group mining claims yew claims located island region long history mining dating back several high grade copper gold mined area geology yew claims found,228
suffering pain depression heartburn well help verified collected one licensed online store great choice wonderful give relief operative support fast shipping secure processing complete confidentiality store verified approved visa,256


In [19]:
data=data[['clean_msg','class']]#.drop_duplicates()
data=data[data['clean_msg']!='']

In [20]:
from sklearn.model_selection import train_test_split
X = data['clean_msg']
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=.2)

In [21]:
vect = CountVectorizer(stop_words='english')
vect.fit(X_train)

# learn training data vocabulary, then use it to create a document-term matrix
X_train_dtm = vect.transform(X_train)

# equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

X_test_dtm= vect.transform(X_test)


In [22]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

print('english dataset Multinomial Naive Bayes ')
print(round(metrics.accuracy_score(y_test, y_pred_class),4))

# metrics.confusion_matrix(y_test, y_pred_class)

english dataset Multinomial Naive Bayes 
0.9621


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train_dtm)
tfidf_transformer.transform(X_train_dtm)
pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])
pipe.fit(X_train, y_train)
print('english dataset tfidtransfomer ')
y_pred = pipe.predict(X_test)
print(round(metrics.accuracy_score(y_test, y_pred),4))
# metrics.confusion_matrix(y_test, y_pred)

english dataset tfidtransfomer 
0.9164


In [17]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='liblinear')

# train the model using X_train_dtm
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_test_dtm)

# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
print('english dataset log regression ')
print(round(metrics.accuracy_score(y_test, y_pred_class),4))
# print(metrics.confusion_matrix(y_test, y_pred_class))
# print(metrics.roc_auc_score(y_test, y_pred_prob))

english dataset log regression 
0.9731


# Chinese

In [59]:
data=pd.read_pickle('ch_emails_raw.pkl')
data=data[['class','message']].drop_duplicates()
data

,class,message
1,ham,讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。\n老领导...
2,spam,尊敬的贵公司(财务/经理)负责人您好！ \n 我是深圳金海实业有限公司（广州...
3,spam,贵公司负责人(经理/财务）您好： \n 深圳市华龙公司受多家公司委托向外低点代开部分增...
4,spam,这是一封HTML格式信件！\n\n-----------------------------...
5,spam,\nTO：贵公司经理、财务\n\n \n 您好！ \n 深圳市春洋贸易有...
...,...,...
64587,ham,清华毕业生在美国失踪续：母子已被证实死亡 \n-----------------------...
64588,ham,她长得很小巧，五官清秀，笑起来很可爱，有点调皮，常捣蛋，是个半大不小的孩子。\n 她平时作...
64593,ham,你似乎在转换话题\n首先，成绩好，推研地把握就大，这个基本上是废话\n每个人都知道\n但是推...
64594,spam,ccert，您好：\n 大多数人靠打工拿工资，用自己的血汗去成就老板的事业，用自己的辛...


In [61]:
# data.groupby('clean_msg').count().sort_values('class')

In [62]:

punc = """！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.- / : 　, !@#$%^&*()_-=[]{ }\|/.,><;:'=+?" "' """
def cnPreprocess(string):
    string=string.lower().replace('\n',' ') #remove newline char
    string=[i for i in string]
    # string=[word.lower() for word in string if len(word)>1]
    string=''.join([word for word in string if word not in 'abcdefghijklmnopqrstuvwxyz '+punc+'0123456789']) # split the string into list and check each word if it is in the english dictionary and longer than 1 alphabet
    return string

In [63]:
data['clean_msg']=data['message'].apply(lambda x: cnPreprocess(x))


In [64]:
data=data[data['clean_msg']!='']

In [65]:
from sklearn.model_selection import train_test_split
X = data['clean_msg']
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=.2)

In [66]:
vect = CountVectorizer(tokenizer=lambda txt:[*txt])
vect.fit(X_train)

# learn training data vocabulary, then use it to create a document-term matrix
X_train_dtm = vect.transform(X_train)

# equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

X_test_dtm= vect.transform(X_test)


C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [67]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

print('chinese dataset Multinomial Naive Bayes ')
print(round(metrics.accuracy_score(y_test, y_pred_class),4))

# metrics.confusion_matrix(y_test, y_pred_class)

chinese dataset Multinomial Naive Bayes 
0.9173


In [68]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train_dtm)
tfidf_transformer.transform(X_train_dtm)
pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])
pipe.fit(X_train, y_train)
print('chinese dataset tfidtransfomer ')
y_pred = pipe.predict(X_test)
print(round(metrics.accuracy_score(y_test, y_pred),4))
# metrics.confusion_matrix(y_test, y_pred)

chinese dataset tfidtransfomer 
0.8348


In [69]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='liblinear')

# train the model using X_train_dtm
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_test_dtm)

# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
print('chinese dataset log regression ')
print(round(metrics.accuracy_score(y_test, y_pred_class),4))
# print(metrics.confusion_matrix(y_test, y_pred_class))
# print(metrics.roc_auc_score(y_test, y_pred_prob))

chinese dataset log regression 
0.9841


C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
